# Alexnet with pretrained +finetunning

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt
import os
import cv2
import time
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import models
import torch.nn.functional as F
import copy
from PIL import Image
import pretrainedmodels.utils as utils
import pretrainedmodels
plt.ion()   # interactive mode
#import config

In [2]:
df_y=pd.read_csv("Datasets/Bronchitis/Train_Labels.csv")

In [3]:
df_y=df_y.sort_values(by=['Images'])
df_y.head()

,Images,Labels
241,CR_0002,0
92,CR_0005,0
503,CR_0006,0
244,CR_0007,0
362,CR_0008,0


In [4]:
pd.value_counts(df_y.Labels)

0    324
1    316
Name: Labels, dtype: int64

In [5]:
labels_y=np.array(df_y.Labels)
labels=labels_y[:200]

In [6]:
import os
import cv2

In [7]:
image_train_path=[]
image_test_path=[]
for path in os.listdir('Datasets/Bronchitis/Train_Images'):
    if 'png'in path:
        image_train_path.append('Datasets/Bronchitis/Train_Images/'+path)
for path in os.listdir('Datasets/Bronchitis/Test_Images'):
    if 'png'in path:
        image_test_path.append('Datasets/Bronchitis/Test_Images/'+path)

images_train=[]
for i in image_train_path[:20]:
    x=cv2.imread(i)
    resized_image = cv2.resize(x, (224,224)) 
    images_train.append(resized_image)


print(images_train[0].shape)

plt.imshow(images_train[0])


In [8]:
#dset_classes_number = 2
#model_ft = models.densenet121(pretrained=True)
#model_ft.classifier = nn.Linear(1024, dset_classes_number)

class Densenet161(nn.Module):
    def __init__(self, num_classes = 2):
        super(Densenet161,self).__init__()
        original_model = models.densenet161()
        self.features = nn.Sequential(*list(original_model.children())[:-1])
        self.classifier = (nn.Linear(2208, num_classes))

    def forward(self, x):
        f = self.features(x)
        f = F.relu(f, inplace=True)
        f = F.avg_pool2d(f, kernel_size=7).view(f.size(0), -1)
        y = self.classifier(f)
        return f,y


x = Densenet161()


inp = Variable(torch.randn(4, 3, 224, 224))

out = x(inp)

out

In [9]:

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

### Directory
data_dir = 'Newbronchdataset'
## loading train and validation image datasetsconvnet
image_datasets = {x: dsets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes


In [10]:
image_datasets['train'].classes

['class0', 'class1']

In [11]:
use_gpu = torch.cuda.is_available()

In [12]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=5):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for data in dataloaders[phase]:
                # get the inputs
                inputs, labels = data

                # wrap them in Variable
                if use_gpu:
                    inputs = Variable(inputs.cuda())
                    labels = Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.data[0] * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [13]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    for i, data in enumerate(dataloaders['val']):
        inputs, labels = data
        if use_gpu:
            inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
        else:
            inputs, labels = Variable(inputs), Variable(labels)

        outputs = model(inputs)
        _, preds = torch.max(outputs.data, 1)

        for j in range(inputs.size()[0]):
            images_so_far += 1
            ax = plt.subplot(num_images//2, 2, images_so_far)
            ax.axis('off')
            ax.set_title('predicted: {}'.format(class_names[preds[j]]))
            imshow(inputs.cpu().data[j])

            if images_so_far == num_images:
                model.train(mode=was_training)
                return
    model.train(mode=was_training)

In [ ]:
#model_ft = models.resnet18(pretrained=True)
model_ft=models.alexnet(pretrained=True)
#num_ftrs = model_ft.fc.in_features
#model_ft.fc = nn.Linear(num_ftrs, 2)
model_ft.classifier  = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 2),
)
#if use_gpu:
#    model_ft = model_ft.cuda()

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=50)

Epoch 0/49
----------
train Loss: 0.7989 Acc: 0.4722
val Loss: 0.7070 Acc: 0.4696

Epoch 1/49
----------
train Loss: 0.7043 Acc: 0.5222
val Loss: 0.7160 Acc: 0.4739

Epoch 2/49
----------
train Loss: 0.6911 Acc: 0.5222
val Loss: 0.7493 Acc: 0.4783

Epoch 3/49
----------
train Loss: 0.6948 Acc: 0.5778
val Loss: 0.7003 Acc: 0.5022

Epoch 4/49
----------
train Loss: 0.6998 Acc: 0.5389
val Loss: 0.7000 Acc: 0.4913

Epoch 5/49
----------
train Loss: 0.6894 Acc: 0.5833
val Loss: 0.6982 Acc: 0.4957

Epoch 6/49
----------
train Loss: 0.6918 Acc: 0.5556
val Loss: 0.7035 Acc: 0.4804

Epoch 7/49
----------
train Loss: 0.6825 Acc: 0.5833
val Loss: 0.7022 Acc: 0.4804

Epoch 8/49
----------
train Loss: 0.6858 Acc: 0.5722


In [ ]:
load_img = utils.LoadImage()
sub=[]
# transformations depending on the model
# rescale, center crop, normalize, and others (ex: ToBGR, ToRange255)
#tf_img = utils.TransformImage(model) 
loader = transforms.Compose([transforms.Scale(224), transforms.ToTensor(),transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
#path_img = 'Datasets/Bronchitis/Test_Images/CR_0001.png'
for each in image_test_path:
    path_image=each
    input_img = load_img(path_img)

    input_tensor = loader(input_img)         # 3x400x225 -> 3x299x299 size may differ
    input_tensor = input_tensor.unsqueeze(0) # 3x299x299 -> 1x3x299x299
    input_i = torch.autograd.Variable(input_tensor,requires_grad=False)

    output_logits = model_ft(input_i)
    a=(output_logits.data).numpy()
    if a[0][0]>a[0][1]:
        sub.append(0)
    else:
        sub.append(1)

In [ ]:
output_logits

In [ ]:
sub

In [ ]:

fil=open('subv1.csv','w')
fil.write('Images,Labels')
for i in range(len(sub)-1):
    fil.write('%s,%d\n'%(image_test_path[i][-11:-7],sub[i]))
fil.write('%s,%d'%(image_test_path[i+1][-11:-7],sub[i+1]))
fil.close()